# LoRA Ablation Study Notebook

Comprehensive comparison of different LoRA ranks

In [ ]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from data.data_loader import CLINC150DataLoader
from models.adaptive_router import create_router_model
import json
import random
import time
from sklearn.metrics import accuracy_score, f1_score

# Set plot style (based on source initialization)
plt.style.use('default')

In [ ]:
class LoRAAblationStudy:
    """Comprehensive LoRA rank ablation study"""

    def __init__(self, output_dir: str = "./ablation_results"):
        self.output_dir = output_dir
        os.makedirs(output_dir, exist_ok=True)

        # LoRA ranks to test [5]
        self.lora_ranks = [2-4]
        self.models_to_test = ["distilbert", "roberta"]

In [ ]:
def _test_single_config(self, model_type: str, lora_rank: int, num_samples: int) -> dict:
    """Test single model configuration"""
    
    random.seed(42)

    # Initialize model with specific LoRA rank [2]
    model = create_router_model(model_type=model_type, lora_rank=lora_rank)

    # Load test data [2]
    data_loader = CLINC150DataLoader(max_length=128)
    test_dataset = data_loader.get_processed_dataset("test")

    # Use subset for faster evaluation [2]
    indices = random.sample(range(len(test_dataset)), min(num_samples, len(test_dataset)))
    subset = test_dataset.select(indices)

    # Predict [6]
    texts = [data_loader.load_dataset("test")[i]["text"] for i in indices]
    true_labels = [subset[i]["labels"] for i in range(len(subset))]
    
    start_time = time.time()
    predictions = model.predict(texts)
    inference_time = time.time() - start_time

    # Calculate metrics [6]
    accuracy = accuracy_score(true_labels, predictions["predictions"])
    f1 = f1_score(true_labels, predictions["predictions"], average="weighted")

    # Count trainable parameters [6, 7]
    trainable_params = sum(p.numel() for p in model.model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.model.parameters())

    return { 
        "model_type": model_type,
        "lora_rank": lora_rank,
        "accuracy": accuracy,
        "f1_score": f1,
        "inference_time_ms": (inference_time / len(texts)) * 1000,
        "trainable_params": trainable_params,
        "total_params": total_params,
        "parameter_efficiency": trainable_params / total_params,
        "samples_tested": len(texts)
    }

In [ ]:
def run_ablation(self, num_samples: int = 1000) -> pd.DataFrame:
    """Run ablation study across models and LoRA ranks""" [5]

    results = []
    for model_type in self.models_to_test:
        print(f"\n🔬 Testing {model_type} with different LoRA ranks...")
        for rank in tqdm(self.lora_ranks, desc=f"{model_type} ranks"):
            try:
                result = self._test_single_config(model_type, rank, num_samples)
                results.append(result) [8]
            except Exception as e:
                print(f"Error testing {model_type} rank {rank}: {e}")
                continue

    # Create results dataframe [8]
    df = pd.DataFrame(results)

    # Save results [8]
    df.to_csv(os.path.join(self.output_dir, "lora_ablation_results.csv"), index=False)

    # Generate plots and analysis [8, 9]
    self._generate_plots(df)
    self._save_detailed_analysis(df)
    
    return df

In [ ]:
def _generate_plots(self, df: pd.DataFrame):
    """Generate comprehensive ablation study plots""" [7]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('LoRA Rank Ablation Study', fontsize=16, fontweight='bold') [10]

    # Plot 1: Accuracy vs LoRA Rank
    ax = axes
    for model_type in df['model_type'].unique():
        model_data = df[df['model_type'] == model_type]
        ax.plot(model_data['lora_rank'], model_data['accuracy'],
                        marker='o', label=model_type, linewidth=2) [10]
    ax.set_xlabel('LoRA Rank')
    ax.set_ylabel('Accuracy')
    ax.set_title('Accuracy vs LoRA Rank')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 2: F1 Score vs LoRA Rank
    ax = axes[1]
    for model_type in df['model_type'].unique():
        model_data = df[df['model_type'] == model_type]
        ax.plot(model_data['lora_rank'], model_data['f1_score'],
                        marker='s', label=model_type, linewidth=2) [3]
    ax.set_xlabel('LoRA Rank')
    ax.set_ylabel('F1 Score')
    ax.set_title('F1 Score vs LoRA Rank')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 3: Inference Time vs LoRA Rank
    ax = axes[5]
    for model_type in df['model_type'].unique():
        model_data = df[df['model_type'] == model_type]
        ax.plot(model_data['lora_rank'], model_data['inference_time_ms'],
                        marker='^', label=model_type, linewidth=2) [11]
    ax.set_xlabel('LoRA Rank')
    ax.set_ylabel('Inference Time (ms)')
    ax.set_title('Inference Time vs LoRA Rank')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 4: Parameter Efficiency
    ax = axes[1]
    for model_type in df['model_type'].unique():
        model_data = df[df['model_type'] == model_type]
        ax.plot(model_data['lora_rank'], model_data['parameter_efficiency'] * 100,
                        marker='d', label=model_type, linewidth=2) [12]
    ax.set_xlabel('LoRA Rank')
    ax.set_ylabel('Parameter Efficiency (%)')
    ax.set_title('Parameter Efficiency vs LoRA Rank')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 5: Trainable Parameters
    ax = axes[1, 1]
    for model_type in df['model_type'].unique():
        model_data = df[df['model_type'] == model_type]
        ax.plot(model_data['lora_rank'], model_data['trainable_params'],
                        marker='v', label=model_type, linewidth=2) [13]
    ax.set_xlabel('LoRA Rank')
    ax.set_ylabel('Trainable Parameters')
    ax.set_title('Trainable Parameters vs LoRA Rank')
    ax.legend()
    ax.grid(True, alpha=0.3)

    # Plot 6: Performance vs Efficiency Trade-off
    ax = axes[1, 5]
    for model_type in df['model_type'].unique():
        model_data = df[df['model_type'] == model_type]
        ax.scatter(model_data['inference_time_ms'], model_data['accuracy'],
                            s=100, alpha=0.7, label=model_type) [9]
        # Add rank annotations
        for _, row in model_data.iterrows():
            ax.annotate(f"r={row['lora_rank']}",
                                        (row['inference_time_ms'], row['accuracy']),
                                        textcoords="offset points", xytext=(5,5),
                                        ha='left', fontsize=8)
    ax.set_xlabel('Inference Time (ms)')
    ax.set_ylabel('Accuracy')
    ax.set_title('Performance vs Efficiency Trade-off')
    ax.legend()
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(self.output_dir, 'lora_ablation_study.png'), dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
def _generate_recommendations(self, df: pd.DataFrame) -> list:
    """Generate practical recommendations from results""" [14]

    recommendations = []
    # Find best configurations
    best_accuracy = df.loc[df['accuracy'].idxmax()]
    best_efficiency = df.loc[df['parameter_efficiency'].idxmax()]
    best_balanced = df.loc[(df['accuracy'] > df['accuracy'].quantile(0.8)) &
                            (df['inference_time_ms'] < df['inference_time_ms'].quantile(0.5))].iloc [14]

    recommendations.extend([
        f"Best Accuracy: {best_accuracy['model_type']} with r={best_accuracy['lora_rank']} "
        f"(Accuracy: {best_accuracy['accuracy']:.4f})", [15]
        f"Most Efficient: {best_efficiency['model_type']} with r={best_efficiency['lora_rank']} "
        f"(Efficiency: {best_efficiency['parameter_efficiency']*100:.1f}%)", [15]
        f"Best Balanced: {best_balanced['model_type']} with r={best_balanced['lora_rank']} "
        f"(Accuracy: {best_balanced['accuracy']:.4f}, Time: {best_balanced['inference_time_ms']:.2f}ms)", [15]
        "Optimal LoRA Rank: r=16 provides best balance of accuracy and efficiency for most models" [4]
    ])
    return recommendations

def _save_detailed_analysis(self, df: pd.DataFrame):
    """Save detailed analysis of results""" [16]
    
    analysis = {
        "summary": {
            "best_overall_accuracy": df['accuracy'].max(),
            "best_config": df.loc[df['accuracy'].idxmax()].to_dict(),
            "fastest_inference": df.loc[df['inference_time_ms'].idxmin()].to_dict(),
            "most_efficient": df.loc[df['parameter_efficiency'].idxmax()].to_dict()
        },
        "recommendations": self._generate_recommendations(df)
    }

    with open(os.path.join(self.output_dir, "detailed_analysis.json"), "w") as f:
        json.dump(analysis, f, indent=2)

    print("\n📋 Ablation Study Recommendations:") [14]
    print("=" * 50)
    for rec in analysis["recommendations"]:
        print(f"• {rec}")


In [ ]:
# Execution Block

print("🔬 Starting LoRA Rank Ablation Study") [4]
print("=" * 50)

# Instantiate and run the study
study = LoRAAblationStudy()
# Using num_samples=2000 as specified in the original main function [4]
results_df = study.run_ablation(num_samples=2000)

print("\n📊 Ablation Study Completed!") [4]
print("Results saved to ./ablation_results/") [4]

# Display the resulting DataFrame
results_df